#**2.4 Synthetic Feasibility**

In [ ]:
import os
os.environ["TF_USE_LEGACY_KERAS"] = "True"
!pip install --pre deepchem
!pip install tf_keras

In [13]:
import deepchem
import warnings
warnings.filterwarnings('ignore')
deepchem.__version__

'2.8.1.dev'

In [15]:
import deepchem as dc
tasks, datasets, transformers = dc.molnet.load_tox21(featurizer='Raw', splitter=None)
molecules = datasets[0].X

In [16]:
from rdkit import Chem
import random
from deepchem.feat import CircularFingerprint
import numpy as np


def create_dataset(fingerprints, smiles_lens, ds_size=100000):
    """
    m1: list of np.Array
        fingerprints for molecules
    m2: list of int
        length of a molecules SMILES string

    returns:
        dc.data.Dataset for input into ScScore Model

    Dataset.X
        shape is (sample_id, molecule_id, features)
    Dataset.y
        shape is (sample_id,)
        values is 1 if the 0th index molecule is more complex
                  0 if the 1st index molecule is more complex
    """
    X, y = [], []
    all_data = list(zip(fingerprints, smiles_lens))
    while len(y) < ds_size:
        i1 = random.randrange(0, len(smiles_lens))
        i2 = random.randrange(0, len(smiles_lens))
        m1 = all_data[i1]
        m2 = all_data[i2]
        if m1[1] == m2[1]:
            continue
        if m1[1] > m2[1]:
            y.append(1.0)
        else:
            y.append(0.0)
        X.append([m1[0], m2[0]])
    return dc.data.NumpyDataset(np.array(X), np.expand_dims(np.array(y), axis=1))

In [17]:
molecule_ds = dc.data.NumpyDataset(np.array(molecules))
splitter = dc.splits.RandomSplitter()
train_mols, test_mols = splitter.train_test_split(molecule_ds)

In [ ]:
n_features = 1024
featurizer = dc.feat.CircularFingerprint(size=n_features, radius=2, chiral=True)
train_features = featurizer.featurize(train_mols.X)
train_smiles_len = [len(Chem.MolToSmiles(x)) for x in train_mols.X]
train_dataset = create_dataset(train_features, train_smiles_len)

In [20]:
model = dc.models.ScScoreModel(n_features=n_features)
model.fit(train_dataset, nb_epoch=20)

ImportError: `keras.optimizers.legacy` is not supported in Keras 3. When using `tf.keras`, to continue using a `tf.keras.optimizers.legacy` optimizer, you can install the `tf_keras` package (Keras 2) and set the environment variable `TF_USE_LEGACY_KERAS=True` to configure TensorFlow to use `tf_keras` when accessing `tf.keras`.

ScScore모델 - 두 분자를 비교하며 학습

SMILES가 더 길수록 복잡한 분자로 가정하고 학습 A,B중 어떤게 더 복잡한 분자인가

분자가 첫번째가 더 길면 1.0 두번째가 더 길면 0.0

일반적으로 SMILES가 길수록 구조가 복잡함 -> 합성난이도 올라감

In [21]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
mol_scores = model.predict_mols(test_mols.X)
smiles_lengths = [len(Chem.MolToSmiles(x)) for x in test_mols.X]

In [23]:
plt.figure(figsize=(20,16))
plt.scatter(smiles_lengths, mol_scores)
plt.xlim(0,80)
plt.xlabel("SMILES length")
plt.ylabel("ScScore")
plt.show()

NameError: name 'smiles_lengths' is not defined

<Figure size 2000x1600 with 0 Axes>

💡 2.4 Synthetic Feasibility(ScScore)


상대적 서열 학습: "이 분자는 80점" 같은 절대적 수치가 아니라, "A보다 B가 만들기에 더 복잡하다'는 선후 관계를 학습시켜 분자 간의 상대적 난이도 지도  구축함.

복잡도의 데이터화: 실습에서는 'SMILES 길이'를 기준으로 삼았지만, 합성 단계 수, 원료 가격, 반응 위험도 등 프로젝트 목적에 맞는 '복잡도'의 정의를 모델에 주입가능

현실적 스크리닝: AI가 설계한 수만 개의 후보 물질 중 물성은 좋지만 제조가 불가능한 것을 사전에 걸러내어, 시간과 비용 낭비를 방지함.